# Serial founder events with migration, founder events and allele frequency variation

In these simulations, the urban environment was colonized through serial founder events beginning from a single rural population at carrying capcity. Simulations varried the initial frequency of both dominant alleles (i.e. _CYP79D15_ and _Li_) in the starting rural population. We performed simulations under 3 migration rates: 0, 0.01, 0.05. Similarly, we used 3 fouding proportions: 0.01, 0.2, 1.0. The parameter combinations were as follows (all varied parameters crossed factorially, total simulations = 81): 

* Number of simulations: 1000
* Number of generations: 500 (following filling of matrix)
* Max Creation probability: 1.0
* Bottleneck proportion: [0.01; 0.2; 1.0]
* pA and pB: [0.1; 0.5; 0.9] and [0.1; 0.5; 0.9]
* Migration rate: [0; 0.01; 0.05]
* Maximum _K_ (rural): 1000
* Minimum _K_ (urban): 1000

In [ ]:
library(plyr)
library(ggplot2)
library(data.table)
library(Rmisc)
library(dplyr)

In [ ]:
#Working directory for datasets varying migration rate and bottleneck proportion 
setwd('/scratch/research/projects/trifolium/SEC_Simulation.Evolutionary.Clines/SEC_Data/drift-migration/1D/AlleleFreq_Vary/one-fill/')

#Load datasets that will be used for analyses
datSlopes <- fread('20171016_SlopeSum_Gen_AlleleFreq-Merged.csv', header = T)
datSlopes$AlleleFreqs <- paste(datSlopes$pA_start, datSlopes$pB_start, sep = "-")
datSlopes$AlleleFreqs <- as.factor(as.character(datSlopes$AlleleFreqs))
datSlopes$AlleleFreqs <- factor(datSlopes$AlleleFreqs, levels=c("0.1-0.1", "0.1-0.5", "0.5-0.1", "0.1-0.9", 
                                                                "0.9-0.1", "0.5-0.5", "0.5-0.9", "0.9-0.5", 
                                                                "0.9-0.9"))

#Data subsets
datSlopes_GenOne <- subset(datSlopes, seq == "1")
# datFreqFirst_bot001_NoMig <- subset(datFreqFirst, bot == "0.01" & Mig_rate == "0")
# datFreqFirst_bot02_NoMig <- subset(datFreqFirst, bot == "0.2" & Mig_rate == "0")
# datFreqFirst_bot001_HighMig <- subset(datFreqFirst, bot == "0.01" & Mig_rate == "0.05")